<a href="https://colab.research.google.com/github/dnaae/minor-internship/blob/FISH-QUANT-pipeline/Cell_level_results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install big-fish

In [ ]:
#imports
import os
import numpy as np
import bigfish
import bigfish.stack as stack
import bigfish.multistack as multistack
import bigfish.plot as plot
from google.colab import drive
print("Big-FISH version: {0}".format(bigfish.__version__))
import pandas as pd
import re

In [ ]:
# Function to mount Google Drive and select a folder
def choose_folder_colab():
    drive.mount('/content/drive')
    drive_folder = "/content/drive/MyDrive/"
    contents = os.listdir(drive_folder)

    print("Contents of your Google Drive:")
    for i, item in enumerate(contents):
        print(f"{i + 1}: {item}")

    while True:
        choice = input("Enter the number of the folder you want to select (q to quit): ")
        if choice.lower() == 'q':
            return None
        try:
            choice = int(choice)
            if 1 <= choice <= len(contents):
                selected_folder = os.path.join(drive_folder, contents[choice - 1])
                return selected_folder
            else:
                print("Invalid choice. Please try again or enter 'q' to quit.")
        except (ValueError, KeyboardInterrupt):
            print("Invalid input. Please try again or enter 'q' to quit.")

In [ ]:
def load_masks_and_data_interactive(output_folder, selected_image_index):
    original_directory = os.getcwd()  # Get the current working directory

    # Construct the path to the "output" subfolder
    output_folder = os.path.join(output_folder, 'output')

    try:
        os.chdir(output_folder)  # Change working directory to "output"

        # Print the list of files in the output folder
        directory_contents = os.listdir()
        print("Contents of the output folder:")
        for item in directory_contents:
            print(item)

        # Generate file names based on the selected image index
        cell_mask_filename = f"cellular_mask_{selected_image_index}.tif"
        nuc_mask_filename = f"nuclear_mask_{selected_image_index}.tif"
        spots_filename = f"spots_cell_{selected_image_index}.csv"
        clusters_filename = f"clusters_cell_{selected_image_index}.csv"

        # Construct paths to the files within the "output" subfolder
        path_cell_label = os.path.join(output_folder, "cellular masks", cell_mask_filename)
        path_nuc_label = os.path.join(output_folder, "nuclear masks", nuc_mask_filename)
        path_spots = os.path.join(output_folder, "notebook 5 output", spots_filename)
        path_foci = os.path.join(output_folder, "notebook 5 output", clusters_filename)

        print("Attempting to load the following files:")
        print(f"Cell mask: {path_cell_label}")
        print(f"Nuclear mask: {path_nuc_label}")
        print(f"Spots file: {path_spots}")
        print(f"Clusters file: {path_foci}")

        if not os.path.exists(path_cell_label) or not os.path.exists(path_nuc_label) or \
                not os.path.exists(path_spots) or not os.path.exists(path_foci):
            print("One or more mask/data files not found in the specified folder.")
            return None, None, None, None

        # Load segmented cells
        cell_label = stack.read_image(path_cell_label)
        print("Segmented cells")
        print("\r shape: {0}".format(cell_label.shape))
        print("\r dtype: {0}".format(cell_label.dtype), "\n")

        # Load segmented nuclei
        nuc_label = stack.read_image(path_nuc_label)
        print("Segmented nuclei")
        print("\r shape: {0}".format(nuc_label.shape))
        print("\r dtype: {0}".format(nuc_label.dtype), "\n")

        # Load detected spots
        spots = stack.read_array_from_csv(path_spots, dtype=np.int64)
        print("Detected spots")
        print("\r shape: {0}".format(spots.shape))
        print("\r dtype: {0}".format(spots.dtype), "\n")

        # Load detected clusters/foci
        # # Load detected clusters/foci
        # if os.path.exists(path_foci):
        #   clusters = stack.read_array_from_csv(path_foci, dtype=np.int64)
        #   print("Detected clusters/foci")
        #   print("\r shape: {0}".format(clusters.shape))
        #   print("\r dtype: {0}".format(clusters.dtype), "\n")
        # else:
        #   clusters = None
        #   print("No clusters found.")

        clusters = stack.read_array_from_csv(path_foci, dtype=np.int64)
        print("Detected clusters/foci")
        print("\r shape: {0}".format(clusters.shape))
        print("\r dtype: {0}".format(clusters.dtype), "\n")
        return cell_label, nuc_label, spots, clusters
    finally:
        os.chdir(original_directory)  # Change working directory back to the original path



In [ ]:
def discriminate_foci_transcription_sites(spots, clusters, nuc_label):
    spots_no_ts, foci, ts = multistack.remove_transcription_site(spots, clusters, nuc_label, ndim=3)
    return spots_no_ts, foci, ts

def identify_objects(spots, nuc_label):
    spots_in, spots_out = multistack.identify_objects_in_region(nuc_label, spots, ndim=3)
    print("detected spots (inside nuclei)")
    print("\r shape: {0}".format(spots_in.shape))
    print("\r dtype: {0}".format(spots_in.dtype), "\n")
    print("detected spots (outside nuclei)")
    print("\r shape: {0}".format(spots_out.shape))
    print("\r dtype: {0}".format(spots_out.dtype))
    return spots_in, spots_out


In [ ]:
def cell_extraction(cell_label, nuc_label, spots_no_ts, foci, ts, image_contrasted, nuc_mip, rna_mip):
    fov_results = multistack.extract_cell(
        cell_label=cell_label,
        ndim=3,
        nuc_label=nuc_label,
        rna_coord=spots_no_ts,
        others_coord={"foci": foci, "transcription_site": ts},
        image=image_contrasted,
        others_image={"dapi": nuc_mip, "smfish": rna_mip}
    )
    print("number of cells identified: {0}".format(len(fov_results)))
    return fov_results

In [ ]:
from skimage.measure import regionprops

In [ ]:
def process_and_plot_cells(fov_results):
    for i, cell_results in enumerate(fov_results):
        print("cell {0}".format(i))

        # get cell results
        cell_mask = cell_results["cell_mask"]
        cell_coord = cell_results["cell_coord"]
        nuc_mask = cell_results["nuc_mask"]
        nuc_coord = cell_results["nuc_coord"]
        rna_coord = cell_results["rna_coord"]
        foci_coord = cell_results["foci"]
        ts_coord = cell_results["transcription_site"]
        image_contrasted = cell_results["image"]
        print("\r number of rna {0}".format(len(rna_coord)))
        print("\r number of foci {0}".format(len(foci_coord)))
        print("\r number of transcription sites {0}".format(len(ts_coord)))

        # plot cell
        plot.plot_cell(
            ndim=3, cell_coord=cell_coord, nuc_coord=nuc_coord,
            rna_coord=rna_coord, foci_coord=foci_coord, other_coord=ts_coord,
            image=image_contrasted, cell_mask=cell_mask, nuc_mask=nuc_mask,
            title="Cell {0}".format(i))


In [ ]:
#+npz
def save_extraction_results(fov_results, path_output, selected_image_index):
    output_folder = os.path.join(path_output, 'output')  # Construct the path to the "output" subfolder
    os.makedirs(output_folder, exist_ok=True)  # Create the "output" subfolder if it doesn't exist

    # Create a new folder "extraction_results_npz_notebook6" inside the "output" directory
    npz_folder = os.path.join(output_folder, "extraction_results_npz_notebook6")
    os.makedirs(npz_folder, exist_ok=True)

    # Iterate over fov_results to modify the 'cell_id' in-place
    for i, cell_results in enumerate(fov_results):
        # Save the results in NPZ format to the new folder
        npz_file_path = os.path.join(npz_folder, f"results_cell_{selected_image_index}.npz")
        stack.save_cell_extracted(cell_results, npz_file_path)
        df_cell = multistack.summarize_extraction_results([cell_results], ndim=3)
        # Modify the 'cell_id' column with the selected_image_index
        df_cell['cell_id'] = selected_image_index
        # Save the DataFrame to a CSV file
        csv_file_path = os.path.join(output_folder, "extraction_results_csv_notebook6", f"extraction_results_{selected_image_index}.csv")
        df_cell.to_csv(csv_file_path, index=False)
        # Summarize results for each cell
        df_cell = multistack.summarize_extraction_results([cell_results], ndim=3)
        # Modify the 'cell_id' column with the selected_image_index
        df_cell['cell_id'] = selected_image_index
        # Save the DataFrame to a CSV file in the existing "extraction_results_csv_notebook6" folder
        #csv_file_path = os.path.join(output_folder, "extraction_results_csv_notebook6", f"extraction_results_{selected_image_index}.csv")
        df_cell.to_csv(csv_file_path, index=False)


In [ ]:
def extract_number_from_filename(filename):
    pattern = re.compile(r'(\d{3})\.tif_C\d\.tif')
    match = re.search(pattern, filename)
    if match:
        return int(match.group(1))
    else:
        return float('inf')

# #for dmso 1 digit
# def extract_number_from_filename(filename):
#     pattern = re.compile(r'(\d{3})\.tif_C\d\.tif')
#     match = re.search(pattern, filename)
#     if match:
#         # Convert the matched group to an integer
#         num = int(match.group(1))
#         # Format the number to have leading zeros and three digits
#         return f"{num:03d}"
#     else:
#         return float('inf')

In [ ]:
#user selects treatment and individual cell images
def process_images_interactively(path_input):
    global nuc_mip, rna_mip, image_contrasted
    output_folder = os.path.join(path_input, 'output')  # Construct the path to the "output" subfolder
    os.chdir(output_folder)  # Change the working directory to the "output" subfolder
    extraction_results_dir = os.path.join(output_folder, "extraction_results_csv_notebook6")
    os.makedirs(extraction_results_dir, exist_ok=True)
    # Now, guide the user in the "output" subfolder
    print(f"Selected folder: {output_folder}")

    # Allow the user to choose a subfolder interactively
    subfolders = [f for f in os.listdir() if os.path.isdir(f)]
    top_level_folder = os.path.basename(os.path.normpath(path_input))
    path_output = os.path.join(path_input, "output")
    if os.path.exists(path_output):
        subfolders = [f for f in os.listdir() if os.path.isdir(f)]
        print("Subfolders:")
        for i, subfolder in enumerate(subfolders):
            print(f"{i + 1}: {subfolder}")

        selected_subfolder_index = int(input("Enter the number of the subfolder you want to select: ")) - 1
        # Iterate through the subfolders
        if 0 <= selected_subfolder_index < len(subfolders):
            selected_subfolder = subfolders[selected_subfolder_index]
            os.chdir(selected_subfolder)  # Change the working directory to the selected subfolder

            # List available channel images
            available_channels = [f for f in os.listdir() if f.endswith("_C1.tif") or f.endswith("_C2.tif")]

            # Separate DAPI and FISH channels
            dapi_channels = [channel for channel in available_channels if "_C1.tif" in channel]
            fish_channels = [channel for channel in available_channels if "_C2.tif" in channel]

            # Extract correct index
            dapi_numbers = [extract_number_from_filename(channel) for channel in dapi_channels]
            for i, (channel, number) in enumerate(zip(dapi_channels, dapi_numbers)):
                print(f"{number} - {channel}")

            dapi_choice = int(input("Enter the number of the DAPI channel you want to select: "))

            # Extract correct index
            fish_numbers = [extract_number_from_filename(channel) for channel in fish_channels]
            for i, (channel, number) in enumerate(zip(fish_channels, fish_numbers)):
                print(f"{number} - {channel}")

            fish_choice = int(input("Enter the number of the FISH channel you want to select: "))

            if dapi_choice in dapi_numbers:
                selected_dapi_channel = dapi_channels[dapi_numbers.index(dapi_choice)]
                print(f"Selected DAPI channel: {selected_dapi_channel}")
                treatment = top_level_folder
                print(f"Selected treatment: {treatment}")

                if fish_choice in fish_numbers:
                    selected_fish_channel = fish_channels[fish_numbers.index(fish_choice)]
                    print(f"Selected FISH channel: {selected_fish_channel}")

                    # Automatically generate file names for channels based on the selected index
                    nuc_filename = selected_dapi_channel
                    rna_filename = selected_fish_channel

                    # Construct the full paths to the images within the "output" subfolder
                    nuc_path = os.path.join(output_folder, selected_subfolder, nuc_filename)
                    rna_path = os.path.join(output_folder, selected_subfolder, rna_filename)

                    if not os.path.exists(nuc_path) or not os.path.exists(rna_path):
                        print("Image files not found in the specified folder.")
                        return

                    # Process the nucleus channel
                    nuc = stack.read_image(nuc_path)
                    nuc_mip = stack.maximum_projection(nuc)
                    nuc_mip=stack.gaussian_filter(nuc_mip, sigma=5)
                    print("Nucleus channel")
                    print("\r shape: {0}".format(nuc.shape))
                    print("\r dtype: {0}".format(nuc.dtype), "\n")

                    # Process the FISH channel
                    rna = stack.read_image(rna_path)
                    rna_mip = stack.maximum_projection(rna)
                    rna_mip=stack.gaussian_filter(rna_mip, sigma=5)
                    print("FISH channel")
                    print("\r shape: {0}".format(rna.shape))
                    print("\r dtype: {0}".format(rna.dtype))

                    # Enhance contrast of the FISH image (rna) and create a 2D maximum projection of the contrast-enhanced image
                    image_contrasted = stack.rescale(rna, channel_to_stretch=0)
                    image_contrasted = stack.maximum_projection(image_contrasted)
                    #image_contrasted = stack.mean_projection(image_contrasted
                    print("Image contrasted successfully.")

                    # Reset the working directory back to the original path
                    os.chdir(path_input)
                else:
                    print("Invalid channel index selected.")

if __name__ == "__main__":
    # Allow the user to choose a folder interactively in Google Colab
    path_input = choose_folder_colab()

    if path_input:
        # The user selected the treatment folder, so guide them in the "output" subfolder
        process_images_interactively(path_input)

        # Pass the selected_image_name to load_masks_and_data_interactive
        selected_image_name = input("Enter the name of the image you want to process: ")
        cell_label, nuc_label, spots, clusters = load_masks_and_data_interactive(path_input, selected_image_name)

        # Now, call the functions to perform the tasks
        spots_no_ts, foci, ts = discriminate_foci_transcription_sites(spots, clusters, nuc_label)
        spots_in, spots_out = identify_objects(spots_no_ts, nuc_label)
        fov_results = cell_extraction(cell_label, nuc_label, spots_no_ts, foci, ts, image_contrasted, nuc_mip, rna_mip)
        # Call the function to process and plot cells
        process_and_plot_cells(fov_results)

        # Call the function to save extraction results
        save_extraction_results(fov_results, path_input, selected_image_name)


In [ ]:
# User selects treatment folder and all images are processed
def process_images_interactively(path_input):
    global nuc_mip, rna_mip, image_contrasted
    output_folder = os.path.join(path_input, 'output')
    os.chdir(output_folder)

    extraction_results_dir = os.path.join(output_folder, "extraction_results_csv_notebook6")
    os.makedirs(extraction_results_dir, exist_ok=True)

    print(f"Selected folder: {output_folder}")

    subfolders = [f for f in os.listdir() if os.path.isdir(f)]
    top_level_folder = os.path.basename(os.path.normpath(path_input))
    path_output = os.path.join(path_input, "output")

    if os.path.exists(path_output):
        subfolders = [f for f in os.listdir() if os.path.isdir(f)]
        print("Subfolders:")
        for i, subfolder in enumerate(subfolders):
            print(f"{i + 1}: {subfolder}")

        selected_subfolder_index = int(input("Enter the number of the subfolder you want to select: ")) - 1

        if 0 <= selected_subfolder_index < len(subfolders):
            selected_subfolder = subfolders[selected_subfolder_index]
            os.chdir(selected_subfolder)

            available_channels = [f for f in os.listdir() if f.endswith("_C1.tif") or f.endswith("_C2.tif")]

            dapi_channels = [channel for channel in available_channels if "_C1.tif" in channel]
            fish_channels = [channel for channel in available_channels if "_C2.tif" in channel]

            for i in range(len(dapi_channels)):
                dapi_channel = dapi_channels[i]
                fish_channel = fish_channels[i]

                dapi_number = extract_number_from_filename(dapi_channel)
                fish_number = extract_number_from_filename(fish_channel)

                print(f"Selected DAPI channel: {dapi_channel}")
                print(f"Selected FISH channel: {fish_channel}")

                treatment = top_level_folder
                print(f"Selected treatment: {treatment}")

                nuc_filename = dapi_channel
                rna_filename = fish_channel

                nuc_path = os.path.join(output_folder, selected_subfolder, nuc_filename)
                rna_path = os.path.join(output_folder, selected_subfolder, rna_filename)

                if not os.path.exists(nuc_path) or not os.path.exists(rna_path):
                    print("Image files not found in the specified folder.")
                    continue

                nuc = stack.read_image(nuc_path)
                nuc_mip_current = stack.maximum_projection(nuc)
                print("Nucleus channel")
                print("\r shape: {0}".format(nuc.shape))
                print("\r dtype: {0}".format(nuc.dtype), "\n")

                rna = stack.read_image(rna_path)
                rna_mip_current = stack.maximum_projection(rna)
                # rna_mip_current= stack.gaussian_filter(rna_mip_current, sigma=5)
                print("FISH channel")
                print("\r shape: {0}".format(rna.shape))
                print("\r dtype: {0}".format(rna.dtype))

                image_contrasted = stack.rescale(rna, channel_to_stretch=0)
                image_contrasted = stack.maximum_projection(image_contrasted)
                #image_contrasted = stack.gaussian_filter(image_contrasted, sigma=5)
                print("Image contrasted successfully.")

                os.chdir(path_input)

                # Load masks and data for the current DAPI-FISH pair
                selected_image_name = f"{dapi_number}"
                cell_label, nuc_label, spots, clusters = load_masks_and_data_interactive(path_input, selected_image_name)

                try:
                    # Update background images for each iteration
                    nuc_mip = nuc_mip_current
                    rna_mip = rna_mip_current

                    # Now, call the functions to perform the tasks
                    spots_no_ts, foci, ts = discriminate_foci_transcription_sites(spots, clusters, nuc_label)
                    spots_in, spots_out = identify_objects(spots_no_ts, nuc_label)
                    fov_results = cell_extraction(cell_label, nuc_label, spots_no_ts, foci, ts, image_contrasted, nuc_mip, rna_mip)

                    # Call the function to process and plot cells
                    process_and_plot_cells(fov_results)

                    # Call the function to save extraction results
                    save_extraction_results(fov_results, path_input, selected_image_name)

                    print(f"Processing complete for DAPI-FISH pair: {dapi_number}-{fish_number}")

                except Exception as e:
                    print(f"Error processing DAPI-FISH pair {dapi_number}-{fish_number}: {e}")

            print("Processing complete for all image channels in the selected subfolder.")

            # Merging CSV files
            output_folder = os.path.join(path_input, 'output')
            extraction_results_dir = os.path.join(output_folder, "extraction_results_csv_notebook6")

            # Get a list of CSV files in the extraction results directory
            csv_files = [f for f in os.listdir(extraction_results_dir) if f.lower().endswith('.csv')]

            if csv_files:
                dataframes = []

                for csv_file in csv_files:
                    # Load each CSV file into a DataFrame
                    file_path = os.path.join(extraction_results_dir, csv_file)
                    try:
                        df = pd.read_csv(file_path, index_col='cell_index')
                    except ValueError:
                        # If 'cell_index' is not a valid index, try reading without specifying the index
                        df = pd.read_csv(file_path)

                    dataframes.append(df)

                # Concatenate all DataFrames into a single DataFrame
                merged_df = pd.concat(dataframes)
                # Save the merged DataFrame to a new CSV file
                csv_filename = os.path.join(output_folder, f'output_dataframe_{top_level_folder}.csv')
                merged_df.to_csv(csv_filename, index=False)
                print(f"Merged DataFrame saved to: {csv_filename}")

        else:
            print("Invalid subfolder index selected.")
    else:
        print("No 'output' folder found in the selected path. Please choose a different folder.")

if __name__ == "__main__":
    path_input = choose_folder_colab()

    if path_input:
        process_images_interactively(path_input)



